In [1]:
%load_ext autoreload
%autoreload 2

import re
from functools import partial


import polars as pl

from mts_ml_cup import urls as u

In [2]:
%%time
urls_popularity = (
    pl.read_parquet("../data/processed/sessions.pq", columns=["user_id", "url_host", "request_cnt"])
    .groupby("url_host")
    .agg(
        [
            pl.col("user_id").n_unique().alias("n_users"),
            pl.col("request_cnt").sum().alias("n_requests"),
            pl.col("url_host").count().alias("n_rows"),
        ]
    )
    .sort(["n_users", "n_rows", "n_requests", "url_host"], reverse=[True, True, True, False])
)

CPU times: user 1min 19s, sys: 18.2 s, total: 1min 37s
Wall time: 34.4 s


In [3]:
urls_popularity

url_host,n_users,n_requests,n_rows
str,u32,i64,u32
"""googleads.g.do...",394562,49094546,22013466
"""yandex.ru""",386405,47494474,19007657
"""avatars.mds.ya...",382692,29760534,16212095
"""i.ytimg.com""",381268,35166640,16901446
"""yastatic.net""",380239,26827614,15053937
"""ad.mail.ru""",374738,29652943,15309099
"""vk.com""",374098,34750259,16695251
"""tpc.googlesynd...",350146,10626800,7294829
"""ads.adfox.ru""",331604,12082356,7771928


In [4]:
urls_popularity.write_csv("../data/processed/urls-popularity.csv")

In [5]:
import joblib as jbl
import cyrtranslit

In [6]:
with open("../data/processed/untitled.txt") as f:
    cities = []
    for row in f:
        try:
            cities.append(row.split("\t")[2])
        except Exception:
            pass
cities = list(map(lambda city: " ".join(city.split("-")), cities))
        
cities_domains = (
    list(map(
        lambda city: "".join(cyrtranslit.to_latin(city.lower(), lang_code="ru").replace("'", "").split()), 
        cities,
    ))
    + list(map(lambda city: "".join(city.lower().split()), cities))
    + [
        "spb", "saintpetersburg", 
        "moscow", "moskovskayaoblast",
        "khb", "nn", 
        "ekb", "akhtubinsk", 
        "arkhangelsk", "artem",
        "astrakhan", "oktyabrsky",
        "samarskayaoblast", "simferopol",
        "евпатория", "vladimirskayaoblast", "leninskkuznetskiy",
        "khantymansiisk", "naberezhnyyechelny", "yakutiya",
        "dolgoprudnyi", "novokuzneck", "novorossiisk", "novorossiysk",
        "serpuhov",
        "yekaterinburg", "samararegion",
        "ulianovsk", "cherepovets",
        "moskovskajaoblast",
        "nizhniynovgorod",
        "zeleznogorsk",
        "velikiynovgorod",
        "gorodets",
        "habarovsk",
        "nizhniynovgorod",
        "makhachkala",
        "nnov",
        "blagoveshchensk",
        "kabardinobalkariya",
        "kabardinobalkarskayarespublika",
        "kaspiysk",
        "khabarovsk",
        "kerch",
        "kirovskayaoblast",
        "kirow", "kirowsk",
        "msk",
        "gornoaltaysk",
    ]
)

In [7]:
accelerators = [
    ".turbopages.org",
    ".cdn.ampproject.org",
]
suffixes = [
    "userapi.com", 
    "online.sberbank.ru", 
    "sbrf.ru", 
    "adriver.ru", 
    "stc.all.kpcdn.net",
    "hh.ru",
    "onlinestars.org",
]
domains = [
    "com", 
    "na",
    "ru", 
    "org", 
    "tilda",
    "ucoz", 
    "info", 
    "net",
    "web",
    "www",
    "lk",
    "my",
    "id",
    "login",
    "connect",
    "amp",
    "lkfl2",
    "android",
    "go",
    "geo",
    "narod",
    "online",
    "forum", "forums",
    "livejournal",
    "herokuapp",
    "api",
    "static",
    "mirtesen",
    "gov",
] + cities_domains
entries = [
    "gde",
    "vsn",
    "cataloxy",
    "promportal",
    "110km",
    "4kolesa",
    "ktogdeest",
    "xiaomi",
    "blizko",
    "bistro",
    "aport",
    "tradedir",
    "lordfilm", 
    "gosuslugi", 
    "mail.yandex",
    "sunlight",
    "dveri", "двери", "двер",
    "auto", "avto", "авто",
    "cerkov", "prihod", "hram", 
    "церковь", "приход", "храм", 
    "eparh", "епарх", "pravoslav", "православ", "eparch", "patriarchia",
    "politeh", "политех",
    "polit", "полит",
    "gruzoperevoz", "грузоперевоз",
    "travel", "путешеств", "tour", "туризм", "trip", "turis",  "турис",
    "econom",
    "karaoke", "караоке",
    "minecraft",
    "tesla",
    "steklo", "стекло",
    "бьюти", "beauty", "красота", "nails",
    "registratura", "регистратура",
    "transport", "транспорт",
    "taxi", "taksi", "такси",
    "vokzal", "вокзал",
    "sushi", "суши",
    "pizza", "пицца",
    "pasport", "passport", "паспорт",
    "fitnes",
    "zoon",
    "football", "футбол",
    "bigbookmed",
    "mama",
    "megafon",
    "miltor",
    "bigbookrelax",
    "dota",
    "naruto",
    "terraria",
    "anime", "аниме",
    "stroy", "строй",
    "remont", "ремонт",
    "woman", "women", "lady", "ladies",
    "кино", "фильм", "movie", "kino", "film",
    "сериал", "serial", "series", "seriahd",
    "rusanalogi",
    "sport", "спорт",
    "school", "школа", "сош",
    "gymnasium", "гимназия", "gymn", "gimnaz", "гимназ",
    "liceum", "лицей", "licey",
    "mebel", "мебел",
    "лицензиироссии",
    "arenda", "аренда",
    "hotel", "отель",
    "timeweb",
    "obam",
    "reginforms", "regtorg",
    "rabota", "работа",
    "tattoo", "тату",
    "recept", "рецепт",
    "ipotek",
    "potolok", "потолок",
    "motor", "мотор",
    "moto", "мото",
    "stomatolog", "стоматолог", "denta", "дента", "zub", "зуб",
    "apteka", "аптека", "аптек",
    "povar", "повар",
    "cvet", "цвет", "flower", "florist",
    "скидк", "skidk",
    "zarplata",
    "razbork",
    "sidex",
    "splar",
    "avia", "авиа",
    "kurort", "курорт",
    "sadovod", "садовод",
    "akvarium", "аквариум",
    "promokod", "промокод",
    "gibdd", "гибдд",
    "bongacams",
    "brawlstars",
    "fortnite",
    "counterstrike",
    "doubleclick",
    "restaurant", "restorant", "ресторан",
    "cafe", "кафе",
    "superjob",
    "cian",
    "ktozvonil",
    "должниковнет",
    "shamora", 
    "nissan",
    "jsprav",
    "prorab", "прораб",
    "spravker",
    "cigar", "сигар",
    "dolg", "долг",
    "nalog", "налог",
    "landshaft",
    "chatruletka",
    "klinik", "клиника", "clinic", "clinik", "klinic",
    "bankiros",
    "vbr", "hipdir",
    "bezformata",
    "rosneft",
    "jobfilter",
    "yurist", "юрист", "advokat", "advocat", "адвокат", "advpalata",
    "ясновидящая",
    "vodokanal", "водоканал",
    "rybalk", "ribalk", "рыбалк", "rybolov", "ribolov", "рыболов",
    "afy",
    "sauna", "сауна",
    "banya", "баня",
    "pulscen",
    "santeh", "santekh", "сантех",
    "dach", "дач", "dacia",
    "bibliotek", "библиотек", "library",
    "radio",
    "радио",
    "tsargrad",
    "chaihona", "чайхана",
    "jobcareer",
    "spravk", "справк",
    "rayon", "raion", "район",
    "hospital", "госпиталь",
    "forex",
    "kosmetolog", "cosmetolog", "косметолог",
    "crypto",
    "unibo",
    "sobak", "собак",
    "kosh",
    "plitka", "плитка",
    "stolova", "столовая",
    "bolnica", "больница",
    "dosaaf", "досааф",
    "doktor", "doctor", "доктор",
    "gdz", "гдз",
    "zoloto", "золото",
    "tele2", "теле2",
    "beeline",
    "namaz",
    "prazdnik", "праздник",
    "torrent", "torent", "торрент",
    "astrolog", "астролог",
    "food",
    "howtonews",
    "phonechecker",
    "detkinclub",
    "aquapark", "akvapark", "аквапарк",
    "gmstar",
    "college", "колледж",
    "roddom", "роддом",
    "pokhude", "pohude", "похуде", "diet", "диет",
    "molodajasemja",
    "corona", "korona", "корона",
    "guitar", "gitar",
    "gotov", "kulinar", "culinar", "кулинар","kylinar",
    "horoscop",
    "mangal", "мангал",
    "rosreestr", "росреестр",
    "manga",
    "medialeks",
    "medicine", "медицин",
    "volvo",
    "mercedes", "мерседесактрос",
    "bmw", "бмв",
    "volkswagen",
    "anek", "анек",
    "renovaciya",
    "standup",
    "lada", "лада",
    "serebro",
    "zapchasti", "запчасти",
    "ruan",
    "chelsea",
    "ohota", "oxota", "охота",
    "legal",
    "kovr", "ковр",
    "mastergym",
    "gym",
    "vkcoffee",
    "coffee", "кофе",
    "cardio", "kardio", "кардио",
    "kalyan", "kalian", "кальян",
    "photo", "фото", "foto",
    "tehnikum", "техникум",
    "novosel",
    "warhammer",
    "pensi", "пенси",
    "pogoda", "weather", "hmn", "meteo",
    "lodk", "boat", "проектстриж", "лодк",
    "koles", "колес",
    "toyota",
    "english",
    "svetofor", "светофор",
    "otpusk",
    "bankrot", "bankrupt", "банкрот",
    "вебслужбы",
    "pohoron", "ritual", "ритуал",
    "webcam",
    "синоним", "synonym",
    "fasad", "facade", "фасад",
    "apart", "апарт",
    "gostinic", "гостиниц",
    "music",
    "pilomaterial", "пиломатериал",
    "psicholog", "psiholog", "psycholog", "psyholog", "psixolog", "psyxolog", "психолог",
    "monastyr", "monastir", "монастыр",
    "podruz", "podrug",
    "mfc", "мфц",
    "islam",
    "pelmen", "пельмен",
    "yaplakal",
    "spravoc", "справоч",
    "postupi",
    "shashl", "шашл",
    "nedvi", "недви",
    "agricultur",
    "zakon", "закон",
    "tablet",
    "lekars", "лекарс",
    "рынок",
    "slovar", "словар",
    "monet", "монет",
    "obrazovan", "образован",
]
mapping = {
    "monastyr": "монастыр", "monastir": "монастыр",
    "pilomaterial": "пиломатериал",
    "apart": "апарт",
    "fasad": "фасад", "facade": "фасад",
    "synonym": "синоним",
    "pohoron": "ритуал", "ritual": "ритуал",
    "bankrot": "банкрот", "bankrupt": "банкрот",
    "svetofor": "светофор",
    "koles": "колес",
    "tehnikum": "техникум",
    "kalyan": "кальян", "kalian": "кальян",
    "kovr": "ковр",
    "ohota": "охота", "oxota": "охота",
    "zapchasti": "запчасти",
    "zoloto": "золото",
    "lada": "лада",
    "bmw": "бмв",
    "anek": "анек",
    "roddom": "роддом",
    "мерседесактрос": "mercedes",
    "medicine": "медицин",
    "mangal": "мангал",
    "gotov": "кулинар", "kulinar": "кулинар", "culinar": "кулинар", "kylinar": "кулинар",
    "guitar": "gitar",
    "corona": "корона",
    "korona": "корона",
    "pokhude": "похуде", "pohude": "похуде", "diet": "похуде", "диет": "похуде",
    "astrolog": "астролог",
    "t.me": "telegram",
    "t.co": "twitter",
    "away.vk": "vk",
    "userapi": "vk",
    "gdz": "гдз",
    "rosreestr": "росреестр",
    "ytimg": "youtube",
    "yastatic": "yandex",
    "aquapark": "аквапарк", "akvapark": "аквапарк",
    "dveri": "двери",
    "двер": "двери",
    "auto": "авто",
    "avto": "авто",
    "torrent": "торрент",
    "torent": "торрент",
    "doktor": "доктор", "doctor": "доктор",
    "prazdnik": "праздник",
    "bolnica": "больница",
    "college": "колледж",
    "cerkov": "церковь",
    "prihod": "церковь", "приход": "церковь",
    "hram": "церковь", "храм": "церковь",
    "eparh": "церковь", "епарх": "церковь", "eparch": "церковь",
    "pravoslav": "церковь", "православ": "церковь", "patriarchia": "церковь",
    "politeh": "политех",
    "polit": "политика",
    "полит": "политика",
    "dosaaf": "досааф",
    "gruzoperevoz": "грузоперевозки",
    "грузоперевоз": "грузоперевозки",
    "travel": "путешествия",
    "путешеств": "путешествия",
    "tour": "путешествия",
    "туризм": "путешествия",
    "trip": "путешествия",
    "turis": "путешествия",
    "турис": "путешествия",
    "econom": "экономика",
    "karaoke": "караоке",
    "steklo": "стекло",
    "бьюти": "красота",
    "beauty": "красота",
    "nails": "красота",
    "registatura": "регистратура",
    "transport": "транспорт",
    "taxi": "такси",
    "taksi": "такси",
    "vokzal": "вокзал",
    "stolova": "столовая",
    "sushi": "суши",
    "pizza": "пицца",
    "pasport": "паспорт",
    "passport": "паспорт",
    "footbal": "футбол",
    "women": "lady",
    "woman": "lady",
    "ladies": "lady",
    "movie": "кино",
    "film": "кино",
    "kino": "кино",
    "фильм": "кино",
    "сериал": "сериалы",
    "serial": "сериалы",
    "seriahd": "сериалы",
    "series": "сериалы",
    "rusanalogi": "аналогилекарств",
    "sport": "спорт",
    "spravoc": "справоч",
    "school": "школа",
    "сош": "школа",
    "mebel": "мебель",
    "мебел": "мебель",
    "arenda": "аренда",
    "гимназ": "гимназия",
    "gymnasium": "гимназия",
    "gymn": "гимназия",
    "gimnaz": "гимназия",
    "hotel": "отель",
    "rabota": "работа",
    "tattoo": "тату",
    "recept": "рецепт",
    "potolok": "потолок",
    "motor": "мотор",
    "moto": "мото",
    "stomatolog": "стоматолог",
    "denta": "стоматолог", "дента": "стоматолог",
    "zub": "стоматолог", "зуб": "стоматолог",
    "apteka": "аптека",
    "аптек": "аптека",
    "cvet": "цвет",
    "flower": "цвет",
    "florist": "цвет",
    "sobak": "собак",
    "skidk": "скидка", "скидк": "скидка",
    "avia": "авиа",
    "kurort": "курорт",
    "liceum": "лицей",
    "licey": "лицей",
    "sadovod": "садовод",
    "promokod": "промокод",
    "hospital": "госпиталь",
    "gibdd": "гибдд",
    "headhunter": "hh",
    "restaurant": "ресторан", "restorant": "ресторан",
    "cafe": "кафе",
    "prorab": "прораб",
    "cigar": "сигар",
    "zakon": "закон",
    "dolg": "долг",
    "nalog": "налог",
    "klinik": "клиника",    "clinic": "клиника", 
    "clinik": "клиника", 
    "klinic": "клиника",
    "yurist": "юрист", "advokat": "юрист", "advocat": "юрист", "адвокат": "юрист", "advpalata": "юрист",
    "vodokanal": "водоканал",
    "rybalk": "рыбалк", "ribalk": "рыбалк",
    "rybolov": "рыбалк", "ribolov": "рыбалк", "рыболов": "рыбалк",
    "sauna": "сауна",
    "banya": "баня",
    "santekh": "сантех",
    "santeh": "сантех",
    "dach": "дач", "dacia": "дач",
    "library": "библиотек",
    "bibliotek": "библиотек",
    "chaihona": "чайхана",
    "spravk": "справк",
    "rayon": "район",
    "raion": "район",
    "kosmetolog": "косметолог", 
    "cosmetolog": "косметолог",
    "plitka": "плитка",
    "mi": "xiaomi",
    "coffee": "кофе",
    "cardio": "кардио",
    "kardio": "кардио",
    "photo": "фото",
    "foto": "фото",
    "pensi": "пенси",
    "pogoda": "погода",
    "weather": "погода",
    "hmn": "погода",
    "meteo": "погода",
    "lodk": "лодк", "boat": "лодк", "проектстриж": "лодк",
    "psicholog": "психолог", "psiholog": "психолог", "psycholog": "психолог", 
    "psyholog": "психолог", "psixolog": "психолог", "psyxolog": "психолог",
    "podruz": "podrug",
    "mfc": "мфц",
    "pelmen": "пельмен",
    "shashl": "шашл",
    "nedvi": "недви",
    "lekars": "лекарс",
    "slovar": "словар",
    "monet": "монет",
    "obrazovan": "образован",
    "tele2": "теле2",
}


url_cleaner = partial(
    u.clean_url,
    preprocessors=[
        u.decode_from_punycode,
        u.lower,
        partial(u.remove_page_accelerator, accelerators=accelerators),
        partial(u.remove_char, chars=["-"]),
        partial(u.save_only_suffix, suffixes=suffixes),
        u.remove_first_level_domain,
        u.remove_one_char_domains,
        partial(u.remove_domains, domains=domains),
        partial(u.save_full_entry, entries=entries),
        partial(u.map_url, mapping=mapping),
        # partial(u.save_regexp, pattern=re.compile("[a-zA-Zа-яА-Я]+")),
    ],
    protected={
        "t.me", "t.co", 
        "паспорт", "транспорт", "мотор", "аналогилекарств", 
        "vkcoffee", "chatruletka",
        "справоч", "-1",
    },
)

In [8]:
urls_popularity["url_host"].head().to_list()

['googleads.g.doubleclick.net',
 'yandex.ru',
 'avatars.mds.yandex.net',
 'i.ytimg.com',
 'yastatic.net',
 'ad.mail.ru',
 'vk.com',
 'tpc.googlesyndication.com',
 'ads.adfox.ru',
 'online.sberbank.ru']

In [9]:
list(map(url_cleaner, urls_popularity["url_host"].head().to_list()))

['doubleclick',
 'yandex',
 'avatars.mds.yandex',
 'youtube',
 'yandex',
 'ad.mail',
 'vk',
 'tpc.googlesyndication',
 'ads.adfox',
 'sberbank']

In [10]:
%%time
urls_popularity = (
    urls_popularity.with_columns(pl.col("url_host").apply(url_cleaner).alias("url_cleaned"))
)

CPU times: user 13.6 s, sys: 83.3 ms, total: 13.7 s
Wall time: 13.7 s


In [11]:
urls_popularity.write_csv("urls.csv")

In [12]:
urls_popularity["url_cleaned"].n_unique()

143983

In [14]:
(
    urls_popularity
    .groupby("url_cleaned")
    .agg(
        [
            pl.col("n_users").sum(),
            pl.col("n_requests").sum(),
            pl.col("n_rows").sum(),
        ]
    )
    .sort(["n_users", "n_rows", "n_requests", "url_cleaned"], reverse=[True, True, True, False])
    .write_csv("../data/processed/urls-popularity-cleaned.csv")
)

In [15]:
cyrtranslit.to_cyrillic("sdelaikamin.ru")

'сделаикамин.ру'

In [16]:
urls_popularity

url_host,n_users,n_requests,n_rows,url_cleaned
str,u32,i64,u32,str
"""googleads.g.do...",394562,49094546,22013466,"""doubleclick"""
"""yandex.ru""",386405,47494474,19007657,"""yandex"""
"""avatars.mds.ya...",382692,29760534,16212095,"""avatars.mds.ya..."
"""i.ytimg.com""",381268,35166640,16901446,"""youtube"""
"""yastatic.net""",380239,26827614,15053937,"""yandex"""
"""ad.mail.ru""",374738,29652943,15309099,"""ad.mail"""
"""vk.com""",374098,34750259,16695251,"""vk"""
"""tpc.googlesynd...",350146,10626800,7294829,"""tpc.googlesynd..."
"""ads.adfox.ru""",331604,12082356,7771928,"""ads.adfox"""


In [883]:
res = (
    urls_popularity
    .groupby("url_cleaned")
    .agg(pl.col("url_host").unique())
    .filter(pl.col("url_host").apply(len) == 1)
    .sort("url_cleaned")
)

In [863]:
res.sample(10)

url_cleaned,url_host
str,list[str]
"""mkm""","[""mkm.ru""]"
"""terskarabian""","[""terskarabian.com""]"
"""teplonasosptz""","[""teplonasos-ptz.ru""]"
"""mescenter""","[""mescenter.ru""]"
"""prestigeomsk""","[""prestige-omsk.ru""]"
"""220voltrurosto...","[""220-voltru-rostov-na-donu.blizko.ru""]"
"""nra""","[""nra.lv""]"
"""mkrep""","[""mkrep.ru""]"
"""zarprod""","[""zarprod.pro""]"


In [23]:
%%time
df = (
    pl.read_parquet("../data/processed/sessions.pq", columns=["url_host", "user_id"])
    .unique()
)

CPU times: user 1min 27s, sys: 19 s, total: 1min 46s
Wall time: 43 s


In [21]:
train_ids = (
    pl.read_parquet("../data/processed/train.pq", columns=["user_id"])
    .with_columns(pl.lit(True).alias("train"))
)
test_ids = (
    pl.read_parquet("../data/processed/test.pq", columns=["user_id"])
    .with_columns(pl.lit(True).alias("test"))
)

In [28]:
uuu = (
    df
    .join(train_ids, how="left", on="user_id")
    .with_columns(pl.col("train").is_not_null())
    .join(test_ids, how="left", on="user_id")
    .with_columns(pl.col("test").is_not_null())
    .filter(pl.col("train") | pl.col("test"))
)

In [30]:
uuu = uuu.join(urls_popularity, how="left", on="url_host")

In [37]:
clusters_info = (
    uuu
    .groupby(["url_cleaned", "train"])
    .agg(
        pl.col("user_id").n_unique().alias("n_users")
    )
    .sort(["url_cleaned", "train", "n_users"], reverse=[False, True, True])
)

In [46]:
test_clusters = set(clusters_info.filter(~pl.col("train"))["url_cleaned"].to_list())
train_clusters = set(clusters_info.filter(pl.col("train"))["url_cleaned"].to_list())

In [48]:
len(train_clusters & test_clusters)

77309

In [52]:
texts = (
    uuu
    .select(["user_id", "url_cleaned"])
    .filter(pl.col("url_cleaned").is_in(list(train_clusters & test_clusters)))
    .groupby("user_id").agg(pl.col("url_cleaned").unique())
    .with_columns(pl.col("url_cleaned").apply(lambda urls: " ".join(urls)))
)

In [57]:
texts.write_parquet("../data/features/url/urls_text-cleaned.pq")

In [54]:
texts["user_id"].n_unique()

414724

In [55]:
uuu["user_id"].n_unique()

414724